<a href="https://colab.research.google.com/github/svenvdpost/natural-computing/blob/project/NC_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
python -m pip install -U pygame==2.4.0 --user

SyntaxError: ignored

In [ ]:
# Working version
import numpy as np
%matplotlib inline
import pylab as plt
import matplotlib.animation as animation
import matplotlib.patches as patches
from IPython.display import Image
from IPython import display
import time

class Boids:
    def __init__(self, 
                 num_boids = 100, 
                 width = 1000, 
                 height = 1000, 
                 alignment_distance = 50, 
                 cohesion_distance = 100, 
                 separation_distance = 25, 
                 vision_distance = None,
                 alignment_strength = 0.1, 
                 cohesion_strength = 0.001, 
                 separation_strength = 0.5, 
                 noise_strength = 0.2,
                 max_velocity = 5):
      
        self.num_boids = num_boids
        self.positions = np.random.uniform(low=[-width/2, -height/2], high=[width/2, height/2], size=(num_boids, 2))
        self.velocities = np.random.uniform(low=[-width/2, -height/2], high=[width/2, height/2], size=(num_boids, 2))
        self.width = width
        self.height = height

        self.alignment_distance = alignment_distance 
        self.cohesion_distance = cohesion_distance 
        self.separation_distance = separation_distance 
        self.alignment_strength = alignment_strength 
        self.cohesion_strength = cohesion_strength 
        self.separation_strength = separation_strength 
        self.noise_strength = noise_strength
        
        if vision_distance != None:
            self.alignment_distance = vision_distance
            self.cohesion_distance = vision_distance
            self.separation_distance = vision_distance
            self.alignment_strength = vision_distance
            self.cohesion_strength = vision_distance
            self.separation_strength = vision_distance


        self.max_velocity = max_velocity 
        self.positions_over_time = [self.positions]
        self.velocities_over_time = [self.velocities]

        # Optimization        
        self.distances = sorted([("alignment", self.alignment_distance), ("cohesion", self.cohesion_distance), ("separation", self.separation_distance)], key=lambda x: x[1])

    def step(self):

        distances = self.get_distances()

        alignment = self.alignment_rule(distances)
        cohesion = self.cohesion_rule(distances)
        separation = self.separation_rule(distances)

        alignment_correction  = (alignment + np.random.uniform(-1,1, (self.num_boids, 2))   * self.noise_strength) * self.alignment_strength
        cohesion_correction   = (cohesion + np.random.uniform(-1,1, (self.num_boids, 2))    * self.noise_strength) * self.cohesion_strength
        separation_correction = (separation + np.random.uniform(-1,1, (self.num_boids, 2)) * self.noise_strength) * self.separation_strength

        self.velocities += alignment_correction + cohesion_correction + separation_correction
        self.limit_velocity()
        self.positions = self.wrap(self.positions + self.velocities)
        self.positions_over_time.append(self.positions.copy())
        self.velocities_over_time.append(self.velocities.copy())
    
    
    def get_distances(self):
        return np.sqrt(np.sum((self.positions[:, np.newaxis] - self.positions) ** 2, axis=2))


    def get_close_boids(self, rule_distance, distances):
        return (distances < rule_distance) & (distances > 0)

    def alignment_rule(self, distances):
        close_boids = self.get_close_boids(self.alignment_distance, distances)
        alignment = np.zeros((len(self.positions), 2))
        for i in range(len(self.positions)):
            neighbors = close_boids[i]
            if any(neighbors):
                alignment[i] = np.mean(self.velocities[neighbors], axis=0)
        return alignment 

    def cohesion_rule(self, distances):
        close_boids = self.get_close_boids(self.cohesion_distance, distances)
        cohesion = np.zeros((len(self.positions), 2))
        for i in range(len(self.positions)):
            neighbors = close_boids[i]
            if any(neighbors):
                cohesion[i] = np.mean(self.positions[neighbors], axis=0) - self.positions[i]
        return cohesion 

    def separation_rule(self, distances):
        close_boids = self.get_close_boids(self.separation_distance, distances)
        separation = np.zeros((len(self.positions), 2))
        for i in range(len(self.positions)):
            neighbors = close_boids[i]
            if any(neighbors):
                separation[i] = np.sum(self.positions[neighbors] - self.positions[i], axis=0)
        return - separation 

    def limit_velocity(self):
        speed = np.sqrt(np.sum(self.velocities ** 2, axis=1))
        too_fast = speed > self.max_velocity
        self.velocities[too_fast] = self.velocities[too_fast] / speed[too_fast, np.newaxis] * self.max_velocity

    def wrap(self, positions):
        positions[:, 0] = np.where(positions[:, 0] > self.width/2, positions[:, 0] - self.width, positions[:, 0])
        positions[:, 0] = np.where(positions[:, 0] < -self.width/2, positions[:, 0] + self.width, positions[:, 0])
        positions[:, 1] = np.where(positions[:, 1] > self.height/2, positions[:, 1] - self.height, positions[:, 1])
        positions[:, 1] = np.where(positions[:, 1] < -self.height/2, positions[:, 1] + self.height, positions[:, 1])
        return positions

    def run_simulation(self, num_steps):
        for i in range(num_steps):
            self.step()
        return self.positions_over_time, self.velocities_over_time


In [6]:
import pygame
from IPython.display import Image, display

pygame.init()
pygame.display.init()
print(pygame.display.list_modes())
screen = pygame.display.set_mode((400, 400))
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.display.flip()
pygame.image.save(screen, 'frame.png')
display(Image(filename='frame.png'))

pygame.quit()

error: ignored

In [ ]:
#TODO Predator class
class Predator:
  def __init__(self, position, velocity, percept)

In [ ]:
#TODO Prey class